<a href="https://colab.research.google.com/github/tousifo/ml_notebooks/blob/main/ALS_QNN_PRO_ACT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cell 1 — Imports & setup for the ALS QNN notebook

**Purpose:** bring in the sklearn utilities, pin/install Qiskit, and import the quantum pieces used by a variational regressor.

### What’s in this cell
- **Data utilities:** `train_test_split`, `MinMaxScaler`
- **Metrics:** `mean_squared_error`, `pearsonr`
- **Install (pinned):** `%pip install qiskit~=1.0 qiskit-machine-learning~=0.8.1 qiskit_algorithms`
- **Quantum stack:** `ZZFeatureMap`, `RealAmplitudes`, `COBYLA`, `VQR`, `Sampler`

> **Note:** If the install line complains about `qiskit_algorithms`, the PyPI package name is often `qiskit-algorithms` (hyphen). We’re not changing your code here—this is just a heads-up.

### Why scale features?
Angle encoders work better when inputs live in a tight range; min–max scaling avoids angle wraparound and makes training steadier.

<details>
<summary>Quick I/O expectations</summary>
After preprocessing later:
- Feature vectors will be scaled to a small range (often [0, 1]).
- The `ZZFeatureMap(feature_dimension=...)` should match your final number of features.
</details>

---


In [1]:
from sklearn.model_selection import train_test_split  # quick train/validation split
from sklearn.preprocessing import MinMaxScaler        # keep features in a compact range for angle encoding
from sklearn.metrics import mean_squared_error        # regression loss (lower is better)
from scipy.stats import pearsonr                      # correlation between predictions and targets (closer to 1 is better)
%pip install qiskit~=1.0 qiskit-machine-learning~=0.8.1 qiskit_algorithms  # pinned install; if it fails, try 'qiskit-algorithms' manually

# Qiskit Imports
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes  # feature map + ansatz for the variational circuit
from qiskit_algorithms.optimizers import COBYLA                  # gradient-free optimizer suited to noisy objectives
from qiskit_machine_learning.algorithms.regressors import VQR    # variational quantum regressor wrapper
from qiskit.primitives import Sampler                            # primitive that evaluates circuits (shot-based)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2


## Cell 2 — CV-safe PRO-ACT preprocessing (`ALSDataProcessor`)

**Purpose:** Turn the raw PRO-ACT CSVs into a clean, cross-validation-safe feature matrix `X` and target `y` (ALSFRS Total slope), then save a single file you can feed into any model (classical or quantum).

### What this cell does
- Anchors timelines to each subject’s **first ALSFRS visit** (t = 0).
- Builds features from the **first 0–90 days** after that anchor across longitudinal tables (ALSFRS, FVC, vitals, labs, grip, muscle).
- Harmonizes **ALSFRS-R** subitems when present (Q10 from 10a; merges Q5a/Q5b).
- Collapses **FVC trials** to the **max per test time** before summarizing.
- Creates **7 summaries** per signal: `min, max, median, std, first, last, slope`.
- Drops columns with **>30% missing**, but does **no scaling/encoding/imputation** here (to avoid leakage).
- Keeps only **eligible subjects**: have an ALSFRS measure **>3 months** and **>12 months** after anchor.
- Saves `final_processed_als_data.csv` and returns a dict with `X`, `y`, `subject_ids`, and the raw joined frame.

### Expected inputs (filenames)
- `PROACT_ALSFRS.csv` *(required)*, plus any of:  
  `PROACT_FVC.csv`, `PROACT_VITALSIGNS.csv`, `PROACT_RILUZOLE.csv`, `PROACT_DEMOGRAPHICS.csv`,  
  `PROACT_LABS.csv`, `PROACT_DEATHDATA.csv`, `PROACT_HANDGRIPSTRENGTH.csv`,  
  `PROACT_MUSCLESTRENGTH.csv`, `PROACT_ALSHISTORY.csv`.

> **Key columns the code looks for**
> - `subject_id` (auto-detected if a similar name exists)
> - Any time field containing **“delta”** or **“day”** (used as days since that table’s baseline)
> - In ALSFRS: `ALSFRS_Total` (numeric)

### Outputs
- **File:** `final_processed_als_data.csv`  
- **Return (dict):**  
  - `X` → features (after 30% missing filter)  
  - `y` → `alsfrs_slope` (per-subject target)  
  - `subject_ids` → subject mapping  
  - `raw_frame` → `[subject_id, alsfrs_slope, X...]` (what gets saved)

> **Windowing:** all longitudinal features are computed over **0–90 days** *from the ALSFRS anchor*.  
> **Slope target:** computed between the **first point after 3 months** and the **first point after 12 months**.

### Gotchas / tips
- If a table has **no time column**, it’s skipped with a warning.
- **Pivoting** of wide/long lab/grip/muscle tables is “best effort” by name patterns; if that fails, it continues safely.
- **No encodings** are done here. Do one-hot/ordinal and scaling **inside your CV folds**.
- If your time unit isn’t in days, ensure the column names include `day`/`delta` or rename before loading.

<details>
<summary>How features are built (quick recipe)</summary>

1) **Anchor** each subject’s rows to the first ALSFRS visit (t=0).  
2) **Align** other tables to ALSFRS time by subtracting each subject’s ALSFRS anchor day.  
3) **Restrict** to days **0–90**.  
4) For every numeric signal per table, compute:  
   `min, max, median, std (ddof=0), first, last, slope(first→last over time)`  
   *(slope is NaN if only one observation or zero time span).*  
5) **Merge** all per-table feature blocks on `subject_id`.
</details>

<details>
<summary>Eligibility logic for y</summary>
A subject is kept if they have **any** ALSFRS measure strictly **>3.0 months** *and* strictly **>12.0 months** after the anchor.  
The slope target uses the very first record after 3 months and the very first after 12 months.
</details>

---

In [2]:
import pandas as pd
import numpy as np
import warnings
from typing import Dict, Optional, List

warnings.filterwarnings("ignore")  # keep notebook output tidy; data has mixed types/old columns


class ALSDataProcessor:
    """
    CV-safe preprocessing for PRO-ACT to reproduce the paper's EDA:
      - Anchor to FIRST ALSFRS visit (t=0) per subject
      - Inputs: first 3 months (0–90 days from anchor) for all longitudinal tables
      - Outcome: ALSFRS Total slope between FIRST-after-3mo and FIRST-after-12mo
      - ALSFRS-R harmonization hooks (Q10 from 10a; merge Q5a/Q5b)
      - FVC reduced to max-of-trials per test before summarization
      - Seven summaries: min, max, median, std, first, last, slope (slope=NaN if only 1 obs)
      - Drop features with >30% missing (no other transforms here — avoid leakage)
    """

    def __init__(self):
        # identifiers/time-like columns we should NOT summarize as numeric features
        self.id_and_delta_cols = {
            "subject_id",
            "alsfrs_delta",
            "fvc_delta",
            "vitals_delta",
            "labs_delta",
            "grip_delta",
            "muscle_delta",
            "onset_delta",
            "death_delta",
            "history_delta",
            "anchor_days",
            "days_from_alsfrs_anchor",
        }

    # --------- Utilities ---------

    @staticmethod
    def _find_time_col(df: pd.DataFrame) -> Optional[str]:
        """Find a time column that represents days since baseline in a table."""
        # Prefer delta
        for c in df.columns:
            lc = c.lower()
            if "delta" in lc:
                return c
        # Fallback to 'days' if present
        for c in df.columns:
            lc = c.lower()
            if "day" in lc:
                return c
        return None

    # --------- ALSFRS-R harmonization ---------

    def _convert_alsfrs_r(self, alsfrs_df: pd.DataFrame) -> pd.DataFrame:
        """
        Prepare ALSFRS table. If ALSFRS-R subitems exist, map per paper:
          - Q10 <- 10a (dyspnea). Ignore 10b/10c.
          - Merge Q5a/Q5b into Q5 if present.
        If only totals exist, this is a no-op aside from coercions.
        """
        df = alsfrs_df.copy()

        if "ALSFRS_Total" in df.columns:
            df["ALSFRS_Total"] = pd.to_numeric(df["ALSFRS_Total"], errors="coerce")

        # Try to locate subitems by loose names
        cols = {c.lower(): c for c in df.columns}

        # Q10 from 10a (dyspnea)
        for candidate in ["alsfrs_r_q10a", "q10a", "dyspnea", "alsfrs_q10a"]:
            if candidate in cols:
                df["Q10"] = pd.to_numeric(df[cols[candidate]], errors="coerce")
                break

        # Merge Q5a/Q5b
        q5a = next(
            (cols[k] for k in ["alsfrs_r_q5a", "q5a", "cutting_wout_gastrostomy"] if k in cols),
            None,
        )
        q5b = next(
            (cols[k] for k in ["alsfrs_r_q5b", "q5b", "cutting_with_gastrostomy"] if k in cols),
            None,
        )
        if q5a and q5b:
            q5a_vals = pd.to_numeric(df[q5a], errors="coerce").values
            q5b_vals = pd.to_numeric(df[q5b], errors="coerce").values
            df["Q5"] = np.nanmax(np.vstack([q5a_vals, q5b_vals]), axis=0)

        return df

    # --------- Anchoring ---------

    def _alsfrs_anchor_days(self, alsfrs_df: pd.DataFrame) -> pd.Series:
        """
        Compute per-subject anchor day = first ALSFRS visit (min delta/days).
        """
        df = alsfrs_df.copy()
        tcol = self._find_time_col(df)
        if tcol is None:
            raise ValueError("ALSFRS table lacks a time delta/days column.")

        df.rename(columns={tcol: "alsfrs_delta"}, inplace=True)
        anchor_map = df.groupby("subject_id")["alsfrs_delta"].min()
        return anchor_map

    # --------- Data I/O ---------

    def load_and_inspect_data(self, file_path: str = "") -> Dict[str, pd.DataFrame]:
        datasets: Dict[str, pd.DataFrame] = {}
        file_list = [
            "PROACT_ALSFRS.csv",
            "PROACT_FVC.csv",
            "PROACT_VITALSIGNS.csv",
            "PROACT_RILUZOLE.csv",
            "PROACT_DEMOGRAPHICS.csv",
            "PROACT_LABS.csv",
            "PROACT_DEATHDATA.csv",
            "PROACT_HANDGRIPSTRENGTH.csv",
            "PROACT_MUSCLESTRENGTH.csv",
            "PROACT_ALSHISTORY.csv",
        ]
        print("--- Loading and Inspecting Data ---")
        for file_name in file_list:
            try:
                df = pd.read_csv(file_path + file_name, on_bad_lines="skip")
                # normalize subject_id
                if "subject_id" not in df.columns:
                    potential = [c for c in df.columns if "subject" in c.lower()]
                    if potential:
                        df = df.rename(columns={potential[0]: "subject_id"})
                # coerce delta-like numeric columns
                for c in df.columns:
                    if "delta" in c.lower() or "day" in c.lower():
                        df[c] = pd.to_numeric(df[c], errors="coerce")
                datasets[file_name] = df
                print(f"✓ {file_name}: {df.shape}")
            except FileNotFoundError:
                print(f"✗ {file_name}: File not found (skipped).")
        return datasets

    # --------- Outcome ---------

    def calculate_alsfrs_slope(self, alsfrs_df: pd.DataFrame) -> pd.DataFrame:
        """
        Outcome = slope between FIRST-after-3mo and FIRST-after-12mo ALSFRS totals,
        with time anchored to first ALSFRS visit.
        """
        df = alsfrs_df.copy()
        tcol = self._find_time_col(df)
        if tcol is None:
            raise ValueError("ALSFRS table lacks a time delta/days column.")
        if "ALSFRS_Total" not in df.columns:
            raise ValueError("ALSFRS_Total missing in ALSFRS table.")

        df.rename(columns={tcol: "alsfrs_delta"}, inplace=True)
        # Anchor
        anchor_map = df.groupby("subject_id")["alsfrs_delta"].min()
        df["days_from_anchor"] = df["alsfrs_delta"] - df["subject_id"].map(anchor_map)
        df["months"] = df["days_from_anchor"] / 30.44

        df = df.sort_values(["subject_id", "months"])
        slopes = {}

        for sid, g in df.groupby("subject_id", sort=False):
            g = g.dropna(subset=["months", "ALSFRS_Total"])
            t1 = g[g["months"] > 3.0].head(1)
            t2 = g[g["months"] > 12.0].head(1)
            if not t1.empty and not t2.empty:
                t1m = float(t1["months"].iloc[0])
                t2m = float(t2["months"].iloc[0])
                t1v = float(t1["ALSFRS_Total"].iloc[0])
                t2v = float(t2["ALSFRS_Total"].iloc[0])
                if t2m > t1m:
                    slopes[sid] = (t2v - t1v) / (t2m - t1m)

        return pd.DataFrame({"subject_id": list(slopes.keys()), "alsfrs_slope": list(slopes.values())})

    # --------- FVC collapse ---------

    @staticmethod
    def _fvc_collapse_trials(df: pd.DataFrame, time_col: str) -> pd.DataFrame:
        """
        Reduce FVC per row/time to the max across trials before summarization.
        Tries to detect typical trial columns; falls back gracefully.
        """
        d = df.copy()
        # Find obvious trial columns
        trial_cols = [c for c in d.columns if "trial" in c.lower()]
        # Some datasets have explicit liters columns per trial name
        if trial_cols:
            d["FVC_Liters"] = pd.to_numeric(d[trial_cols].max(axis=1), errors="coerce")
            keep = ["subject_id", time_col, "FVC_Liters"]
            return d[keep]
        # Fallbacks: look for liters column names
        liter_like = [c for c in d.columns if "liter" in c.lower() or "fvc" in c.lower()]
        if liter_like:
            # If multiple, take row-wise max
            d["FVC_Liters"] = pd.to_numeric(d[liter_like].max(axis=1), errors="coerce")
            keep = ["subject_id", time_col, "FVC_Liters"]
            return d[keep]
        # Last resort: return as-is
        return d

    # --------- Longitudinal summarization ---------

    def create_longitudinal_features(self, df: pd.DataFrame, time_col: str, prefix: str) -> pd.DataFrame:
        """
        Create 7 summaries over [0, 90] days from ALSFRS anchor:
          min, max, median, std, first, last, slope(first→last)
        Slope remains NaN if only one observation or zero time span.
        """
        if time_col not in df.columns:
            return pd.DataFrame()

        d = df.copy()
        # Coerce numerics (but keep subject_id/time cols)
        for c in d.columns:
            if c not in {"subject_id", time_col}:
                d[c] = pd.to_numeric(d[c], errors="coerce")

        # Ensure window is 0..90 days from ALSFRS anchor (already anchored)
        d = d[(d[time_col] >= 0) & (d[time_col] <= 90)].copy()
        if d.empty:
            return pd.DataFrame()

        # Value columns (exclude identifiers/derived delta/time)
        val_cols = [
            c
            for c in d.select_dtypes(include=[np.number]).columns
            if c not in self.id_and_delta_cols and c not in {"subject_id", time_col}
        ]
        if not val_cols:
            return pd.DataFrame()

        out = []
        g = d.groupby("subject_id", as_index=True)
        for col in val_cols:
            agg = g[col].agg(["min", "max", "median", "first", "last"])
            std_ = g[col].std(ddof=0).rename("std")
            slope = g.apply(
                lambda x: (x[col].iloc[-1] - x[col].iloc[0]) / max(1e-9, (x[time_col].iloc[-1] - x[time_col].iloc[0]))
                if len(x) > 1 and (x[time_col].iloc[-1] - x[time_col].iloc[0]) > 0
                else np.nan
            ).rename("slope")
            feat = pd.concat([agg, std_, slope], axis=1)
            feat.columns = [f"{prefix}{col}_{cname}" for cname in feat.columns]
            out.append(feat)

        return pd.concat(out, axis=1).reset_index()

    # --------- Static table processing (no encoding here to avoid leakage) ---------

    @staticmethod
    def process_static_data(df: pd.DataFrame) -> pd.DataFrame:
        """
        CV-safe: DO NOT encode here. Just keep one row per subject.
        (Do categorical encoding in your modeling pipeline.)
        """
        if "subject_id" not in df.columns:
            return pd.DataFrame()
        # Keep first non-duplicated row per subject_id
        return df.drop_duplicates(subset=["subject_id"]).copy()

    # --------- Merge features ---------

    def merge_all_features(self, datasets: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        if "PROACT_DEMOGRAPHICS.csv" not in datasets:
            raise ValueError("Demographics file is missing.")

        # Build ALSFRS anchor map
        alsfrs = datasets["PROACT_ALSFRS.csv"]
        anchor_map = self._alsfrs_anchor_days(alsfrs)

        # Start with demographics (static)
        final_df = self.process_static_data(datasets["PROACT_DEMOGRAPHICS.csv"])

        # Add static-ish other tables (keep CV-safe; no encodings)
        for file in ["PROACT_RILUZOLE.csv", "PROACT_ALSHISTORY.csv"]:
            if file in datasets:
                static_df = self.process_static_data(datasets[file])
                final_df = pd.merge(final_df, static_df, on="subject_id", how="left")

        # Longitudinal configs
        longitudinal = {
            "PROACT_ALSFRS.csv": "alsfrs_",
            "PROACT_FVC.csv": "fvc_",
            "PROACT_VITALSIGNS.csv": "vitals_",
            "PROACT_LABS.csv": "labs_",
            "PROACT_HANDGRIPSTRENGTH.csv": "grip_",
            "PROACT_MUSCLESTRENGTH.csv": "muscle_",
        }

        print("\n--- Generating Longitudinal Features (anchored to first ALSFRS; window = 0–90 days) ---")
        for file, prefix in longitudinal.items():
            if file not in datasets:
                continue

            df = datasets[file].copy()
            tcol = self._find_time_col(df)
            if tcol is None:
                print(f"Warning: No time delta/days column in {file}. Skipping.")
                continue

            # Anchor this table to ALSFRS first visit
            df["anchor_days"] = df["subject_id"].map(anchor_map)
            df = df[~df["anchor_days"].isna()].copy()
            df["days_from_alsfrs_anchor"] = pd.to_numeric(df[tcol], errors="coerce") - df["anchor_days"]

            # FVC special handling: collapse to max-of-trials BEFORE summarization
            if file == "PROACT_FVC.csv":
                df = self._fvc_collapse_trials(df, time_col="days_from_alsfrs_anchor")

            # Attempt to pivot long-form measurement tables (best effort)
            if file in {"PROACT_LABS.csv", "PROACT_MUSCLESTRENGTH.csv", "PROACT_HANDGRIPSTRENGTH.csv"}:
                try:
                    test_cols = [
                        c
                        for c in df.columns
                        if c not in {"subject_id", "days_from_alsfrs_anchor", "anchor_days"}
                        and any(k in c.lower() for k in ["test", "exam", "muscle", "site", "name", "strength_test"])
                    ]
                    value_cols = [
                        c
                        for c in df.columns
                        if c not in {"subject_id", "days_from_alsfrs_anchor", "anchor_days"}
                        and any(k in c.lower() for k in ["result", "value", "strength", "score"])
                    ]
                    if test_cols and value_cols:
                        tcol_name = test_cols[0]
                        vcol_name = value_cols[0]
                        df[vcol_name] = pd.to_numeric(df[vcol_name], errors="coerce")
                        df = (
                            df.pivot_table(
                                index=["subject_id", "days_from_alsfrs_anchor"],
                                columns=tcol_name,
                                values=vcol_name,
                                aggfunc="mean",
                            )
                            .reset_index()
                        )
                except Exception as e:
                    print(f"Warning: Pivoting failed for {file}: {e}")

            feats = self.create_longitudinal_features(df, "days_from_alsfrs_anchor", prefix)
            if not feats.empty:
                final_df = pd.merge(final_df, feats, on="subject_id", how="left")

        return final_df

    # --------- Eligibility ---------

    def filter_eligible_patients(self, feature_df: pd.DataFrame, alsfrs_df: pd.DataFrame) -> pd.DataFrame:
        """
        Keep subjects who have ANY ALSFRS >3 months AND >12 months AFTER the ALSFRS anchor.
        """
        df = alsfrs_df.copy()
        tcol = self._find_time_col(df)
        if tcol is None:
            raise ValueError("ALSFRS table lacks a time delta/days column.")

        df.rename(columns={tcol: "alsfrs_delta"}, inplace=True)
        anchor_map = df.groupby("subject_id")["alsfrs_delta"].min()
        df["days_from_anchor"] = df["alsfrs_delta"] - df["subject_id"].map(anchor_map)
        df["months"] = df["days_from_anchor"] / 30.44

        g = df.groupby("subject_id")["months"]
        has_t1 = g.apply(lambda s: (s > 3.0).any())
        has_t2 = g.apply(lambda s: (s > 12.0).any())
        eligible_ids = has_t1[has_t1].index.intersection(has_t2[has_t2].index)

        print(f"\nEligible patients: {len(eligible_ids)} / {df['subject_id'].nunique()}")
        return feature_df[feature_df["subject_id"].isin(eligible_ids)].copy()

    # --------- Orchestration ---------

    def run_pipeline(self, file_path: str = "") -> Optional[Dict[str, pd.DataFrame]]:
        """
        End-to-end EDA (CV-safe) that writes 'final_processed_als_data.csv'.
        No imputation/scaling/feature selection here — do that inside your CV pipeline.
        """
        print("====== Starting ALS Data Preprocessing Pipeline ======")
        datasets = self.load_and_inspect_data(file_path)
        if "PROACT_ALSFRS.csv" not in datasets:
            print("CRITICAL ERROR: PROACT_ALSFRS.csv not found. Aborting.")
            return None

        # ALSFRS prep + anchor
        datasets["PROACT_ALSFRS.csv"] = self._convert_alsfrs_r(datasets["PROACT_ALSFRS.csv"])

        # Outcome
        target_df = self.calculate_alsfrs_slope(datasets["PROACT_ALSFRS.csv"])
        print(f"\nCalculated ALSFRS slope for {len(target_df)} patients.")

        # Features
        full_features = self.merge_all_features(datasets)

        # Eligibility
        eligible_features = self.filter_eligible_patients(full_features, datasets["PROACT_ALSFRS.csv"])

        # Join features + target
        final_df = pd.merge(eligible_features, target_df, on="subject_id", how="inner")

        # Drop features with >30% missing
        print("\n--- Handling Missing Values (Dropping cols with >30% missing) ---")
        initial_cols = len(final_df.columns)
        missing_thresh = 0.30
        min_non_na = int(np.ceil(len(final_df) * (1 - missing_thresh)))
        final_df = final_df.dropna(axis=1, thresh=min_non_na)
        dropped = initial_cols - len(final_df.columns)
        print(f"Dropped {dropped} columns for >{int(missing_thresh*100)}% missingness.")

        # Separate X/y (no transforms here to avoid leakage)
        if "alsfrs_slope" not in final_df.columns:
            print("No target available after merges. Aborting.")
            return None

        y = final_df["alsfrs_slope"]
        valid = y.notna()
        final_df = final_df.loc[valid].reset_index(drop=True)

        subject_ids = final_df["subject_id"]
        y = final_df["alsfrs_slope"]
        X = final_df.drop(columns=["subject_id", "alsfrs_slope"])

        # Save CV-safe engineered dataset (raw features)
        out = pd.concat([subject_ids, y, X], axis=1)
        out.to_csv("final_processed_als_data.csv", index=False)
        print("\n✅ Saved CV-safe engineered data to 'final_processed_als_data.csv'")
        print(f"Feature matrix shape: {X.shape} | Target length: {len(y)}")

        return {"X": X, "y": y, "subject_ids": subject_ids, "raw_frame": out}


if __name__ == "__main__":
    # If your CSVs live elsewhere, set file_path accordingly (e.g., "C:/data/PROACT/")
    file_path = ""
    processor = ALSDataProcessor()
    processed = processor.run_pipeline(file_path=file_path)
    if processed is not None:
        print("\nPreview of columns:", list(processed["X"].columns)[:10])
        print("Done.")


====== Starting ALS Data Preprocessing Pipeline ======
--- Loading and Inspecting Data ---
✓ PROACT_ALSFRS.csv: (73845, 20)
✓ PROACT_FVC.csv: (49110, 10)
✓ PROACT_VITALSIGNS.csv: (84721, 36)
✓ PROACT_RILUZOLE.csv: (10363, 3)
✓ PROACT_DEMOGRAPHICS.csv: (12504, 14)
✓ PROACT_LABS.csv: (2937162, 5)
✓ PROACT_DEATHDATA.csv: (5043, 3)
✓ PROACT_HANDGRIPSTRENGTH.csv: (19032, 11)
✓ PROACT_MUSCLESTRENGTH.csv: (204875, 10)
✓ PROACT_ALSHISTORY.csv: (13765, 16)

Calculated ALSFRS slope for 1897 patients.

--- Generating Longitudinal Features (anchored to first ALSFRS; window = 0–90 days) ---

Eligible patients: 3317 / 8538

--- Handling Missing Values (Dropping cols with >30% missing) ---
Dropped 1413 columns for >30% missingness.

✅ Saved CV-safe engineered data to 'final_processed_als_data.csv'
Feature matrix shape: (1897, 346) | Target length: 1897

Preview of columns: ['Demographics_Delta', 'Age', 'Race_Caucasian', 'Sex', 'Subject_used_Riluzole', 'Riluzole_use_Delta', 'Subject_ALS_History_Delta'

## Cell 5 — FAST classical baselines (successive halving + pipeline caching)

**Purpose:** Train strong-but-quick RF & SVR baselines using **successive halving** (prunes weak configs early) and **joblib cache** (reuses preprocessing across CV folds).

### What this cell does
- Loads `final_processed_als_data.csv`, quick **80/20** split, optional shuffle for fold homogeneity.
- Auto-detects **numeric vs categorical** columns.
- Builds **in-pipeline** preprocessing (impute; scaling only for SVR; OHE for categoricals).
- Runs **HalvingGridSearchCV (cv=3)** for:
  - **RandomForestRegressor** (no scaling needed).
  - **SVR (RBF)** (with scaling).
- Evaluates on the held-out test set and prints **RMSE, PCC** with **95% bootstrap CIs**.
- Prints a quick **RF+SVR 50–50 ensemble** as a sanity check.

### Why it’s fast
- **Successive halving** cuts off underperformers early → far fewer total fits.
- **joblib.Memory** caches transformers inside the pipeline → repeated CV splits don’t recompute imputation/encoding from scratch.

### Outputs (printed)
- Best params for RF & SVR (FAST grids).
- **Test Set Performance (FAST mode)** table with RMSE & PCC + CIs.
- Optional **RF+SVR Ensemble** metrics.

> Tip: If you’re only benchmarking speed, comment out the SVR block; RF alone is often a strong baseline here.

---


In [3]:
import numpy as np
import pandas as pd
from typing import Tuple, Dict
import warnings
warnings.filterwarnings("ignore")

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

# Halving search (successive halving)
from sklearn.experimental import enable_halving_search_cv  # noqa: F401
from sklearn.model_selection import HalvingGridSearchCV

# caching
from joblib import Memory

np.random.seed(42)

# ---------- Metrics ----------
def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def safe_pcc(y_true, y_pred) -> float:
    yt = np.asarray(y_true, dtype=float).ravel()
    yp = np.asarray(y_pred, dtype=float).ravel()
    if yt.std() < 1e-12 or yp.std() < 1e-12:
        return 0.0
    return float(np.corrcoef(yt, yp)[0, 1])

def bootstrap_ci(y_true, y_pred, metric_fn, n_boot=800, alpha=0.95, seed=42) -> Tuple[float, float]:
    rng = np.random.default_rng(seed)
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    n = len(y_true)
    stats = []
    idx = np.arange(n)
    for _ in range(n_boot):
        b = rng.choice(idx, size=n, replace=True)
        stats.append(metric_fn(y_true[b], y_pred[b]))
    lo = float(np.percentile(stats, (1 - alpha) / 2 * 100))
    hi = float(np.percentile(stats, (1 + alpha) / 2 * 100))
    return lo, hi

# ---------- Main ----------
def run_classical_pipeline_fast() -> pd.DataFrame:
    print("====== FAST Classical Baselines (successive halving, cached) ======")

    # 1) Load engineered data
    df = pd.read_csv("final_processed_als_data.csv")
    print(f"✓ Loaded engineered dataset: {df.shape}")

    X = df.drop(columns=["subject_id", "alsfrs_slope"])
    y = df["alsfrs_slope"].astype(float)

    # Optional quick shuffle for better fold homogeneity
    X, y = shuffle(X, y, random_state=42)

    # 80/20 split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )
    print(f"Split: train={X_train.shape[0]}, test={X_test.shape[0]}")

    # 2) Column typing
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
    print(f"Detected numeric={len(num_cols)}, categorical={len(cat_cols)}")

    # Pipeline cache
    memory = Memory(location="sk_cache", verbose=0)

    # Preprocessors
    # Numeric: impute → (optional scaler in SVR branch)
    num_rf = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ], memory=memory)

    num_svr = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ], memory=memory)

    if len(cat_cols) > 0:
        cat_common = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ], memory=memory)
        preproc_rf = ColumnTransformer(
            transformers=[("num", num_rf, num_cols), ("cat", cat_common, cat_cols)],
            remainder="drop"
        )
        preproc_svr = ColumnTransformer(
            transformers=[("num", num_svr, num_cols), ("cat", cat_common, cat_cols)],
            remainder="drop"
        )
    else:
        # No categoricals → simpler (faster) preprocessors
        preproc_rf = num_rf
        preproc_svr = num_svr

    # 3) Pipelines (with small but effective grids)
    rf_pipe = Pipeline(steps=[
        ("preprocess", preproc_rf),
        ("select", SelectKBest(score_func=f_regression, k="all")),
        ("model", RandomForestRegressor(random_state=42, n_jobs=-1))
    ], memory=memory)

    rf_grid: Dict[str, list] = {
        # keep imputer fixed (median) to avoid recomputing transforms
        "select__k": ["all", 50],              # feature count toggle
        "model__n_estimators": [250],          # enough trees, faster than 500
        "model__max_depth": [None, 12],
        "model__min_samples_leaf": [1, 2],
        "model__max_features": ["sqrt"],       # stable setting
    }

    # Successive halving (aggressive elimination reduces fits)
    rf_search = HalvingGridSearchCV(
        rf_pipe,
        rf_grid,
        factor=3,
        resource="n_samples",
        min_resources="exhaust",
        cv=3,
        scoring="neg_root_mean_squared_error",   # optimize RMSE directly
        n_jobs=-1,
        verbose=0,
        refit=True
    )
    print("\n--- Fitting RandomForest (HalvingGridSearchCV, cv=3) ---")
    rf_search.fit(X_train, y_train)
    print(f"RF best params: {rf_search.best_params_}")

    # SVR (trimmed grid; if you need even faster, comment this whole block)
    svr_pipe = Pipeline(steps=[
        ("preprocess", preproc_svr),
        ("select", SelectKBest(score_func=f_regression, k="all")),
        ("model", SVR(kernel="rbf"))
    ], memory=memory)

    svr_grid: Dict[str, list] = {
        "select__k": ["all", 50],
        "model__C": [1.0, 3.0],
        "model__epsilon": [0.1],
        "model__gamma": ["scale"],
    }

    svr_search = HalvingGridSearchCV(
        svr_pipe,
        svr_grid,
        factor=3,
        resource="n_samples",
        min_resources="exhaust",
        cv=3,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1,
        verbose=0,
        refit=True
    )
    print("\n--- Fitting SVR (HalvingGridSearchCV, cv=3) ---")
    svr_search.fit(X_train, y_train)
    print(f"SVR best params: {svr_search.best_params_}")

    # 4) Test-set evaluation + (faster) bootstrap CIs
    results = []

    for name, est in [("Random Forest", rf_search), ("SVR (RBF)", svr_search)]:
        y_pred = est.best_estimator_.predict(X_test)
        test_rmse = rmse(y_test, y_pred)
        test_pcc  = safe_pcc(y_test.values, y_pred)

        rmse_lo, rmse_hi = bootstrap_ci(y_test.values, y_pred, rmse, n_boot=800, alpha=0.95, seed=123)
        pcc_lo,  pcc_hi  = bootstrap_ci(y_test.values, y_pred, safe_pcc, n_boot=800, alpha=0.95, seed=456)

        results.append({
            "Model": name,
            "RMSE": test_rmse,
            "RMSE 95% CI Low": rmse_lo,
            "RMSE 95% CI High": rmse_hi,
            "PCC": test_pcc,
            "PCC 95% CI Low": pcc_lo,
            "PCC 95% CI High": pcc_hi,
        })

    results_df = pd.DataFrame(results).set_index("Model")
    print("\n====== Test Set Performance (FAST mode) ======")
    print(results_df.round(4))

    # Optional quick 50–50 blend (no extra CV)
    rf_pred = rf_search.best_estimator_.predict(X_test)
    svr_pred = svr_search.best_estimator_.predict(X_test)
    ens_pred = 0.5 * (rf_pred + svr_pred)

    ens_rmse = rmse(y_test, ens_pred)
    ens_pcc  = safe_pcc(y_test.values, ens_pred)
    ens_rmse_ci = bootstrap_ci(y_test.values, ens_pred, rmse, n_boot=800, alpha=0.95, seed=789)
    ens_pcc_ci  = bootstrap_ci(y_test.values, ens_pred, safe_pcc, n_boot=800, alpha=0.95, seed=101112)

    print("\n--- Simple RF+SVR Avg Ensemble (FAST) ---")
    print(pd.DataFrame({
        "RMSE": [ens_rmse],
        "RMSE 95% CI Low": [ens_rmse_ci[0]],
        "RMSE 95% CI High": [ens_rmse_ci[1]],
        "PCC": [ens_pcc],
        "PCC 95% CI Low": [ens_pcc_ci[0]],
        "PCC 95% CI High": [ens_pcc_ci[1]],
    }, index=["RF+SVR Ensemble"]).round(4))

    return results_df


if __name__ == "__main__":
    run_classical_pipeline_fast()


====== FAST Classical Baselines (successive halving, cached) ======
✓ Loaded engineered dataset: (1897, 348)
Split: train=1517, test=380
Detected numeric=343, categorical=3

--- Fitting RandomForest (HalvingGridSearchCV, cv=3) ---
RF best params: {'model__max_depth': None, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 2, 'model__n_estimators': 250, 'select__k': 50}

--- Fitting SVR (HalvingGridSearchCV, cv=3) ---
SVR best params: {'model__C': 1.0, 'model__epsilon': 0.1, 'model__gamma': 'scale', 'select__k': 'all'}

====== Test Set Performance (FAST mode) ======
                 RMSE  RMSE 95% CI Low  RMSE 95% CI High     PCC  \
Model                                                              
Random Forest  0.5905           0.5467            0.6386  0.1918   
SVR (RBF)      0.5907           0.5405            0.6404  0.2131   

               PCC 95% CI Low  PCC 95% CI High  
Model                                           
Random Forest          0.0909           0.2972  


## Cell 4 — Variational Quantum Regressor (VQC) trained with SPSA

**Purpose:** End-to-end quantum regressor that predicts ALSFRS slope from engineered features.  
It builds a circuit **ZZFeatureMap → EfficientSU2**, measures the **average Z** observable, and fits a tiny linear head `ŷ = α·⟨O⟩ + β`. Parameters **θ (circuit), α, β** are trained with **SPSA** on MSE using an Aer/Estimator backend.

### Pipeline (quick map)
1) **Load** `final_processed_als_data.csv` → keep **numeric** columns only.  
2) **Train/test split** (80/20).  
3) **Feature pick** inside train: **RF top-K** numeric features (default **K=16**).  
4) **Train/val split** within training set for **early stopping**.  
5) **Impute + Standardize** (train-only fit) → **PLSRegression** → reduce to `n_qubits`.  
6) **Min-max to angles** `[0, π]` per component (angle encoding).  
7) **Circuit**: `ZZFeatureMap(n_qubits, reps) ∘ EfficientSU2(n_qubits, reps, entanglement="linear")`.  
8) **Observable**: mean of Z on each qubit: \( O = \frac{1}{n}\sum_i Z_i \).  
9) **Train with SPSA** on MSE, early stop on `val_rmse + (1 - val_pcc)`.  
10) **Test**: RMSE, PCC + **95% bootstrap CIs**.

### Key knobs (you can pass via `run_vqc`)
- `n_qubits` (e.g., 4) & `pls_components` (**must match** `n_qubits`).
- `topk` features before PLS (default 16).
- Circuit depth: `fmap_reps`, `ansatz_reps`.
- SPSA schedule: `a, c, A, alpha, gamma`; steps & `batch_size`.
- Early stopping `patience`. Backend auto-selects **AerEstimator** if available.

> **Tip:** For faster smoke tests, try `spsa_steps=150` and `batch_size=64`.

<details>
<summary>SPSA in 30 seconds</summary>
At step *k*, build **one random ± perturbation** of the whole parameter vector, compute two losses `L+` and `L−`, estimate a stochastic gradient with a finite difference, and apply a scheduled step size. It needs **2 objective calls per step**, independent of parameter count.
</details>

<details>
<summary>Encoding & Observable</summary>
- **Encoding:** PLS projects scaled features into `n_qubits` components, then min-max maps them into **[0, π]** angles to feed the `ZZFeatureMap`.  
- **Observable:** ⟨O⟩ is between −1 and 1; the linear head `(α, β)` rescales it to the target range.
</details>

<details>
<summary>Gotchas</summary>
- **`pls_components == n_qubits`** is asserted.  
- If **qiskit-aer** isn’t present, it falls back to the built-in `Estimator` (slower).  
- `topk` must be ≤ number of numeric features in the training split.  
- Seeds are set for numpy and RF; quantum backends may still introduce sampling noise.
</details>

**Printed outputs:** split sizes, top-K list, live train/val metrics during SPSA, and final test **RMSE/PCC + 95% CIs**.

---


In [4]:
%pip install pennylane pennylane-lightning[gpu] torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 7.1 MB/s eta 0:00:00


In [9]:
import os, warnings, numpy as np, pandas as pd
from tqdm.auto import tqdm
warnings.filterwarnings("ignore")

# keep CPU stable
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

import torch
torch.set_num_threads(1)
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pennylane as qml
from pennylane.qnn import TorchLayer

from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

# ---------------- utils ----------------
np.random.seed(42); torch.manual_seed(42)

def rmse(y_true, y_pred): return float(np.sqrt(mean_squared_error(y_true, y_pred)))
def safe_pcc(a, b):
    a, b = np.asarray(a).ravel(), np.asarray(b).ravel()
    if a.std()==0 or b.std()==0: return 0.0
    v = pearsonr(a, b)[0]
    return float(v) if np.isfinite(v) else 0.0

def load_als_data(path="final_processed_als_data.csv"):
    df = pd.read_csv(path)
    X = df.drop(columns=["subject_id", "alsfrs_slope"], errors="ignore")
    y = df["alsfrs_slope"].astype(float).values
    m = ~np.isnan(y)
    X, y = X.loc[m].reset_index(drop=True), y[m]
    print(f"✓ Loaded data: X={X.shape}, y={y.shape}")
    print(f"Target stats: mean={y.mean():.3f}, std={y.std():.3f}, range=[{y.min():.3f},{y.max():.3f}]")
    return X, y

def sanitize_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if df[c].dtype == object:
            try: df[c] = pd.to_numeric(df[c])
            except Exception: df[c] = pd.factorize(df[c].astype(str))[0]
    return df

def select_features(X_df, y, k=8):
    """RF importance + correlation blend (train-only)."""
    imp = SimpleImputer(strategy="median")
    Xn = imp.fit_transform(X_df)
    rf = RandomForestRegressor(n_estimators=350, random_state=42, n_jobs=-1).fit(Xn, y)
    rf_s = rf.feature_importances_
    corr = np.array([abs(np.corrcoef(Xn[:, i], y)[0,1]) if Xn[:, i].std()>0 else 0.0 for i in range(Xn.shape[1])])
    def nz(v): m=v.max(); return v/(m+1e-8) if m>0 else v
    score = 0.7*nz(rf_s) + 0.3*nz(corr)
    idx = np.argsort(score)[::-1][:k]
    cols = [X_df.columns[i] for i in idx]
    print(f"✓ Selected top-{k}: {cols}")
    return idx.tolist(), cols

def oof_baseline(X, y, model="hgb", n_splits=5, seed=42):
    """OOF baseline (full features), plus final refit model for holdouts."""
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof = np.zeros(len(y), dtype=float)

    if model == "hgb":
        def new_model():
            return HistGradientBoostingRegressor(
                loss="squared_error",
                learning_rate=0.06,
                max_depth=None,
                max_bins=255,
                l2_regularization=0.0,
                max_iter=400,
                random_state=seed
            )
    else:
        def new_model():
            return RandomForestRegressor(
                n_estimators=500, max_features="sqrt", min_samples_leaf=2,
                random_state=seed, n_jobs=-1
            )

    for tr, va in kf.split(X):
        mdl = new_model().fit(X[tr], y[tr])
        oof[va] = mdl.predict(X[va])

    mdl_full = new_model().fit(X, y)
    return oof, mdl_full

# ---------------- dataset ----------------
class ResidualDataset(Dataset):
    def __init__(self, X, base_pred, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.base = torch.tensor(base_pred, dtype=torch.float32).view(-1,1)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self): return len(self.y)
    def __getitem__(self, i): return self.X[i], self.base[i], self.y[i]

# ---------------- model ----------------
class ResidualQNN(nn.Module):
    """
    Classical encoder -> quantum core (trainable encoding + re-uploading) -> head.
    Head learns residual over a strong baseline: y ≈ base + residual_qnn
    """
    def __init__(self, in_dim, n_qubits=4, n_layers=2, hidden=16, dropout=0.10, diff_method="backprop"):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers

        # classical encoder to [-1,1] angles
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, 24), nn.ReLU(),
            nn.Linear(24, n_qubits), nn.Tanh()
        )

        dev = qml.device("default.qubit", wires=n_qubits, shots=None)

        # observables: X_i, Z_i, and ring ZZ_(i,i+1)  → 3*n_qubits outputs
        X_ops = [qml.PauliX(i) for i in range(n_qubits)]
        Z_ops = [qml.PauliZ(i) for i in range(n_qubits)]
        ZZ_ops= [qml.PauliZ(i) @ qml.PauliZ((i+1)%n_qubits) for i in range(n_qubits)]
        observables = X_ops + Z_ops + ZZ_ops

        @qml.qnode(dev, interface="torch", diff_method=diff_method)
        def qnode(inputs, weights, alpha, beta):
            # batch-safe encoding
            angles = inputs if inputs.ndim == 2 else inputs[None, :]
            # initial trainable encoding
            for i in range(n_qubits):
                qml.RY(np.pi * (alpha[i]*angles[..., i] + beta[i]), wires=i)
            # L blocks
            for l in range(n_layers):
                for i in range(n_qubits):
                    qml.RY(weights[l, i, 0], wires=i)
                    qml.RZ(weights[l, i, 1], wires=i)
                for i in range(n_qubits):
                    qml.CNOT(wires=[i, (i+1)%n_qubits])
                # re-upload with learned scale/shift
                for i in range(n_qubits):
                    qml.RY(0.5*np.pi * (alpha[i]*angles[..., i] + beta[i]), wires=i)
            return [qml.expval(op) for op in observables]

        weight_shapes = {
            "weights": (n_layers, n_qubits, 2),
            "alpha":   (n_qubits,),
            "beta":    (n_qubits,)
        }
        self.quantum = TorchLayer(qnode, weight_shapes)

        out_dim = 3 * n_qubits  # X + Z + ZZ
        self.head = nn.Sequential(
            nn.Linear(out_dim + 1, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, 1)
        )

    def forward(self, x, base_pred):
        angles = self.encoder(x)
        qfeat  = self.quantum(angles)          # (B, 3*n_qubits)
        h      = torch.cat([qfeat, base_pred], dim=1)
        return self.head(h).squeeze(1)

# ---------------- train / eval ----------------
def train_one_epoch(model, loader, opt, loss_fn, clip=1.0, device="cpu"):
    model.train(); tot=0.0
    for xb, baseb, yb in tqdm(loader, desc="Train", leave=False):
        xb, baseb, yb = xb.to(device), baseb.to(device), yb.to(device)
        opt.zero_grad()
        out = model(xb, baseb)
        loss = loss_fn(out, yb)
        loss.backward()
        if clip: nn.utils.clip_grad_norm_(model.parameters(), clip)
        opt.step()
        tot += loss.item() * xb.size(0)
    return tot/len(loader.dataset)

@torch.no_grad()
def eval_epoch(model, loader, loss_fn, device="cpu"):
    model.eval(); tot=0.0; pred=[]; tgt=[]
    for xb, baseb, yb in loader:
        xb, baseb, yb = xb.to(device), baseb.to(device), yb.to(device)
        out = model(xb, baseb)
        loss = loss_fn(out, yb)
        tot += loss.item() * xb.size(0)
        pred.append(out.cpu().numpy()); tgt.append(yb.cpu().numpy())
    pred = np.concatenate(pred); tgt = np.concatenate(tgt)
    return tot/len(loader.dataset), pred, tgt

# ---------------- pipeline ----------------
def run_qnn_residual(
    data_path="final_processed_als_data.csv",
    n_features=8,
    n_qubits=4,
    n_layers=2,
    epochs=25,
    batch_size=128,
    patience=7,
    lr=1.5e-2,
    weight_decay=1e-4,
    mc_samples=1,
    baseline_type="hgb",      # "hgb" or "rf"
    diff_method="backprop"    # "backprop" works great for these tiny circuits
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # -------- data
    X_raw, y = load_als_data(data_path)
    X_raw = sanitize_features(X_raw)

    # splits
    X_tr_raw, X_te_raw, y_tr, y_te = train_test_split(X_raw, y, test_size=0.2, random_state=42)
    X_tr_raw, X_va_raw, y_tr, y_va = train_test_split(X_tr_raw, y_tr, test_size=0.125, random_state=123)
    print(f"Splits: train={len(y_tr)}, val={len(y_va)}, test={len(y_te)}")

    # -------- feature selection (train-only) for QNN
    feat_idx, feat_cols = select_features(pd.DataFrame(X_tr_raw, columns=X_raw.columns), y_tr, k=n_features)

    # QNN branch: impute+scale top-k features
    imp_q = SimpleImputer(strategy="median")
    sc_q  = RobustScaler()
    Xtr_q = sc_q.fit_transform(imp_q.fit_transform(pd.DataFrame(X_tr_raw, columns=X_raw.columns).iloc[:, feat_idx])).astype(np.float32)
    Xva_q = sc_q.transform(imp_q.transform(pd.DataFrame(X_va_raw, columns=X_raw.columns).iloc[:, feat_idx])).astype(np.float32)
    Xte_q = sc_q.transform(imp_q.transform(pd.DataFrame(X_te_raw, columns=X_raw.columns).iloc[:, feat_idx])).astype(np.float32)

    # Baseline branch: impute full features (NO scaling needed for RF/HGB)
    imp_b = SimpleImputer(strategy="median")
    Xtr_b = imp_b.fit_transform(pd.DataFrame(X_tr_raw, columns=X_raw.columns)).astype(np.float32)
    Xva_b = imp_b.transform(pd.DataFrame(X_va_raw, columns=X_raw.columns)).astype(np.float32)
    Xte_b = imp_b.transform(pd.DataFrame(X_te_raw, columns=X_raw.columns)).astype(np.float32)

    # -------- strong OOF baseline on FULL features
    oof_tr, base_tr_full = oof_baseline(Xtr_b, y_tr, model=baseline_type, n_splits=5, seed=42)
    y_base_va = base_tr_full.predict(Xva_b)

    # residual targets
    res_tr = y_tr - oof_tr
    res_va = y_va - y_base_va
    mu_res, sd_res = res_tr.mean(), res_tr.std() + 1e-8
    z_tr = (res_tr - mu_res)/sd_res
    z_va = (res_va - mu_res)/sd_res

    tr_ld = DataLoader(ResidualDataset(Xtr_q, oof_tr, z_tr), batch_size=batch_size, shuffle=True)
    va_ld = DataLoader(ResidualDataset(Xva_q, y_base_va, z_va), batch_size=batch_size, shuffle=False)

    # -------- model
    model = ResidualQNN(
        in_dim=n_features, n_qubits=n_qubits, n_layers=n_layers,
        hidden=16, dropout=0.10, diff_method=diff_method
    ).to(device)
    print(f"Training Residual QNN on device={device} — features={feat_cols} — obs_out={3*n_qubits}")

    loss_fn = nn.SmoothL1Loss(beta=0.5)   # Huber, robust
    opt     = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    sch     = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=max(epochs, 50))

    best, bad, best_state = np.inf, 0, None
    for ep in range(1, epochs+1):
        tr_loss = train_one_epoch(model, tr_ld, opt, loss_fn, clip=1.0, device=device)
        va_loss, _, _ = eval_epoch(model, va_ld, loss_fn, device=device)
        tqdm.write(f"[{ep:02d}] train={tr_loss:.5f}  val={va_loss:.5f}  lr={sch.get_last_lr()[0]:.4g}")
        if va_loss + 1e-8 < best:
            best, bad = va_loss, 0
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        else:
            bad += 1
            if bad >= patience:
                tqdm.write(f"Early stopping at epoch {ep} (patience={patience}).")
                break
        sch.step()

    if best_state is not None:
        model.load_state_dict(best_state)

    # -------- final baseline on train+val (FULL features) for test
    if baseline_type == "hgb":
        base_all = HistGradientBoostingRegressor(
            loss="squared_error", learning_rate=0.06, max_iter=500,
            max_bins=255, random_state=777
        ).fit(np.vstack([Xtr_b, Xva_b]), np.hstack([y_tr, y_va]))
    else:
        base_all = RandomForestRegressor(
            n_estimators=600, max_features="sqrt", min_samples_leaf=2,
            random_state=777, n_jobs=-1
        ).fit(np.vstack([Xtr_b, Xva_b]), np.hstack([y_tr, y_va]))

    y_base_te = base_all.predict(Xte_b)

    # test residuals (MC-dropout optional)
    te_ld = DataLoader(ResidualDataset(Xte_q, y_base_te, np.zeros_like(y_base_te)),
                       batch_size=batch_size, shuffle=False)
    model.eval()
    with torch.no_grad():
        if mc_samples <= 1:
            _, z_pred, _ = eval_epoch(model, te_ld, loss_fn, device=device)
            res_pred = z_pred * sd_res + mu_res
            y_qnn = y_base_te + res_pred
        else:
            preds = []
            for _ in range(mc_samples):
                for m in model.modules():
                    if isinstance(m, nn.Dropout): m.train()
                _, z_pred, _ = eval_epoch(model, te_ld, loss_fn, device=device)
                preds.append(z_pred)
            z_mean = np.mean(preds, axis=0)
            res_pred = z_mean * sd_res + mu_res
            y_qnn = y_base_te + res_pred

    # metrics
    rf_rmse, rf_pcc, rf_r2 = rmse(y_te, y_base_te), safe_pcc(y_te, y_base_te), r2_score(y_te, y_base_te)
    q_rmse,  q_pcc,  q_r2  = rmse(y_te, y_qnn),     safe_pcc(y_te, y_qnn),     r2_score(y_te, y_qnn)

    print("\n===== TEST METRICS =====")
    print(f"Baseline ({baseline_type}) → RMSE={rf_rmse:.4f}  PCC={rf_pcc:.4f}  R²={rf_r2:.4f}")
    print(f"Residual QNN              → RMSE={q_rmse:.4f}  PCC={q_pcc:.4f}  R²={q_r2:.4f}")
    print(f"ΔRMSE (base − QNN)       → {rf_rmse - q_rmse:+.4f}  ({'QNN better' if (rf_rmse-q_rmse)>0 else 'Base better'})")

    return dict(rmse=q_rmse, pcc=q_pcc, r2=q_r2,
                rmse_base=rf_rmse, pcc_base=rf_pcc, r2_base=rf_r2,
                features=feat_cols)

if __name__ == "__main__":
    _ = run_qnn_residual(
        data_path="final_processed_als_data.csv",
        n_features=8,      # 8 or 10 usually best on CPU
        n_qubits=4,        # keep 4 for speed
        n_layers=2,        # try 3 if you have more time
        epochs=25,         # early stop will kick in
        batch_size=128,
        patience=7,
        lr=1.5e-2,
        weight_decay=1e-4,
        mc_samples=1,      # set 3 for a tiny bump
        baseline_type="hgb",   # "hgb" stronger than "rf"
        diff_method="backprop"
    )


✓ Loaded data: X=(1897, 346), y=(1897,)
Target stats: mean=-0.667, std=0.572, range=[-3.628,1.208]
Splits: train=1327, val=190, test=380
✓ Selected top-8: ['alsfrs_ALSFRS_Total_slope', 'fvc_FVC_Liters_slope', 'alsfrs_ALSFRS_Total_std', 'fvc_FVC_Liters_std', 'alsfrs_Q1_Speech_min', 'alsfrs_Q1_Speech_last', 'alsfrs_Q3_Swallowing_slope', 'vitals_Pulse_first']
Training Residual QNN on device=cpu — features=['alsfrs_ALSFRS_Total_slope', 'fvc_FVC_Liters_slope', 'alsfrs_ALSFRS_Total_std', 'fvc_FVC_Liters_std', 'alsfrs_Q1_Speech_min', 'alsfrs_Q1_Speech_last', 'alsfrs_Q3_Swallowing_slope', 'vitals_Pulse_first'] — obs_out=12


Train:   0%|          | 0/11 [00:00<?, ?it/s]

[01] train=0.55934  val=0.56807  lr=0.015


Train:   0%|          | 0/11 [00:00<?, ?it/s]

[02] train=0.55726  val=0.56206  lr=0.01499


...

[11] train=0.53617  val=0.55715  lr=0.01357


Train:   0%|          | 0/11 [00:00<?, ?it/s]

[12] train=0.53686  val=0.55695  lr=0.01328


Train:   0%|          | 0/11 [00:00<?, ?it/s]

[13] train=0.53150  val=0.56505  lr=0.01297
Early stopping at epoch 13 (patience=7).

===== TEST METRICS =====
Baseline (hgb) → RMSE=0.6020  PCC=0.2746  R²=0.0526
Residual QNN              → RMSE=0.5950  PCC=0.2881  R²=0.0745
ΔRMSE (base − QNN)       → +0.0070  (QNN better)
